In [ ]:
! nvidia-smi

Wed Nov 29 17:01:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip install transformers accelerate optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from accelerate.utils import release_memory
import torch
import time

In [ ]:
from transformers import set_seed
set_seed(42)

In [ ]:
def model_memory_footprint(model):
    print(f"{model.get_memory_footprint() / 1024 / 1024 / 1024:.2f} GB")

Normal Model - FP32

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-1b7",
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-1b7")

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [ ]:
model

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 2048)
    (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
  )
  (

In [ ]:
model.num_parameters()

1722408960

In [ ]:
model_memory_footprint(model)

6.42 GB


```
1B Params = 1,000,000,000 * 32 bits

= (10^9)*32/8/1024/1024/1024
= 3.7252902985 GB
```

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
prompt = """The cat, commonly referred to as the domestic cat or house cat, is"""

In [ ]:
def generate_helper(pipeline, prompt):
    # warm up
    for i in range(3):
      _ = pipeline("Warm up")

    # measure latency in a simple way
    start = time.time()
    out = pipeline(prompt, max_new_tokens=128)
    end = time.time()

    generated_text = out[0]["generated_text"][len(prompt):]

    latency_per_token_in_ms = ((end-start)/len(pipeline.tokenizer(generated_text)["input_ids"]))*1000

    print(f"Generated in {time.time() - start} seconds.")

    # return the generated text and the latency
    return {"text": out[0]["generated_text"], "latency": f"{round(latency_per_token_in_ms,2)}ms/token"}

In [ ]:
res = generate_helper(pipe, prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated in 4.035271406173706 seconds.


In [ ]:
print(f"Latency: {res['latency']}")
print(f"GPU memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Generated Code:\n{res['text']}")

Latency: 31.52ms/token
GPU memory: 6.42 GB
Generated Code:
The cat, commonly referred to as the domestic cat or house cat, is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized


In [ ]:
model_memory_footprint(model)

6.42 GB


In [ ]:
release_memory(model)

del pipe
del model

import gc
import torch

gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

## Float 16

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-1b7",
    device_map="auto",
    torch_dtype=torch.bfloat16, # BFLOAT16
)
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-1b7")

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
res = generate_helper(pipe, prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated in 7.050082445144653 seconds.


In [ ]:
print(f"Latency: {res['latency']}")
print(f"GPU memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Generated Code:\n{res['text']}")

Latency: 55.07ms/token
GPU memory: 9.63 GB
Generated Code:
The cat, commonly referred to as the domestic cat or house cat, is a medium-sized cat that is native to the Americas. The cat is a member of the Felidae family, which includes the dog, the wolf, the coyote, the raccoon, the opossum, the rabbit, the rat, the mouse, the hedgehog, the squirrel, the skunk, the opossum, the rat, the mouse, the hedgehog, the squirrel, the skunk, the opossum, the rat, the mouse, the hedgehog, the squirrel, the skunk, the opossum, the rat, the mouse, the hedgehog,


In [ ]:
model_memory_footprint(model)

3.21 GB


In [ ]:
release_memory(model)

del pipe
del model

import gc
import torch

gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

## Flash Attention

Will only work on Ampere, Ada, or Hopper GPUs (e.g., A100, RTX 3090, RTX 4090, H100).

In [ ]:
from optimum.bettertransformer import BetterTransformer

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-1b7",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-1b7")

In [ ]:
BetterTransformer.transform(model, keep_original_model=False)

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 2048)
    (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttentionLayerBetterTransformer(
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e-05, elementwis

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
res = generate_helper(pipe, prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


RuntimeError: ignored

In [ ]:
print(f"Latency: {res['latency']}")
print(f"GPU memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Generated Code:\n{res['text']}")

Expected Output:

```
/home/inkers/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Generated in 2.468855142593384 seconds.
Latency: 19.28ms/token
GPU memory: 3.22 GB
Generated Code:
The cat, commonly referred to as the domestic cat or house cat, is a medium-sized cat that is native to the Americas. The cat is a member of the Felidae family, which includes the dog, the wolf, the coyote, the raccoon, the opossum, the rabbit, the rat, the mouse, the hedgehog, the squirrel, the skunk, the opossum, the rat, the mouse, the hedgehog, the squirrel, the skunk, the opossum, the rat, the mouse, the hedgehog, the squirrel, the skunk, the opossum, the rat, the mouse, the hedgehog,
```

In [ ]:
model_memory_footprint(model)

In [ ]:
release_memory(model)

del pipe
del model

import gc
import torch

gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

## Int8 Quantization

In [ ]:
! pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 2.4 MB/s eta 0:00:00


Restart Runtime after installation

In [ ]:
from transformers import BitsAndBytesConfig

In [ ]:
eight_bit_config = BitsAndBytesConfig(
    load_in_8bit=True
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-1b7",
    device_map="auto",
    quantization_config=eight_bit_config
)
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-1b7")

In [ ]:
model_memory_footprint(model)

2.08 GB


In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
res = generate_helper(pipe, prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated in 14.353595733642578 seconds.


In [ ]:
print(f"Latency: {res['latency']}")
print(f"GPU memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Generated Code:\n{res['text']}")

Latency: 112.13ms/token
GPU memory: 2.09 GB
Generated Code:
The cat, commonly referred to as the domestic cat or house cat, is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized


## Int4 Quantization

In [ ]:
four_bit_config = BitsAndBytesConfig(
    load_in_4bit=True
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-1b7",
    device_map="auto",
    quantization_config=four_bit_config
)
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-1b7")

In [ ]:
model_memory_footprint(model)

1.52 GB


In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
res = generate_helper(pipe, prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated in 6.459570646286011 seconds.


In [ ]:
print(f"Latency: {res['latency']}")
print(f"GPU memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Generated Code:\n{res['text']}")

Latency: 50.46ms/token
GPU memory: 3.69 GB
Generated Code:
The cat, commonly referred to as the domestic cat or house cat, is a small, medium-sized feline. It is the largest of the feline species, with a body length of between 30 and 40 cm. The cat is a medium-sized feline, with a body length of between 30 and 40 cm. The cat is a medium-sized feline, with a body length of between 30 and 40 cm. The cat is a medium-sized feline, with a body length of between 30 and 40 cm. The cat is a medium-sized feline, with a body length of between 30 and 40 cm. The cat is a medium-sized feline, with a body length of between 30 and 40 cm. The cat is a medium-sized


## 4bit with NF4

In [ ]:
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-1b7",
    device_map="auto",
    quantization_config=nf4_config
)
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-1b7")

In [ ]:
model_memory_footprint(model)

1.52 GB


In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
res = generate_helper(pipe, prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated in 10.191665172576904 seconds.


In [ ]:
print(f"Latency: {res['latency']}")
print(f"GPU memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Generated Code:\n{res['text']}")

Latency: 79.58ms/token
GPU memory: 3.14 GB
Generated Code:
The cat, commonly referred to as the domestic cat or house cat, is a small, medium-sized feline. It is the largest of the feline species, with a body length of between 30 and 40 cm. The cat is a medium-sized feline, with a body length of between 30 and 40 cm. The cat is a medium-sized feline, with a body length of between 30 and 40 cm. The cat is a medium-sized feline, with a body length of between 30 and 40 cm. The cat is a medium-sized feline, with a body length of between 30 and 40 cm. The cat is a medium-sized feline, with a body length of between 30 and 40 cm. The cat is a medium-sized


## CPU Offloading

In [ ]:
cpu_eight_bit_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-1b7",
    device_map="auto",
    quantization_config=cpu_eight_bit_config
)
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-1b7")

In [ ]:
model_memory_footprint(model)

2.08 GB


In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
res = generate_helper(pipe, prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated in 15.530627965927124 seconds.


In [ ]:
print(f"Latency: {res['latency']}")
print(f"GPU memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Generated Code:\n{res['text']}")

Latency: 121.33ms/token
GPU memory: 5.22 GB
Generated Code:
The cat, commonly referred to as the domestic cat or house cat, is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized feline species native to the Americas. The domestic cat is a medium-sized
